In [78]:
import pandas as pd
import datetime
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.inspection import permutation_importance
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

from sklearn.tree import export_graphviz
from graphviz import Source
import numpy as np
import optuna
from sklearn.model_selection import TimeSeriesSplit
import xgboost as xgb

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

from statsmodels.tsa.stattools import acf
from sklearn.linear_model import LassoCV
from sklearn.tree import export_graphviz
from graphviz import Source
import torch.optim as optim

### Prepare data

In [33]:
tr_1 = pd.read_csv("/kaggle/input/transaction-ds/transactions_1.csv", index_col=0)
tr_2 = pd.read_csv("/kaggle/input/transaction-ds/transactions_2.csv", index_col=0)
tr_total = pd.concat([tr_1, tr_2], axis = 0).sort_values(by = ["date"])
tr_total = tr_total.drop_duplicates()

In [34]:
# convert date column to datetime
tr_total['date'] = pd.to_datetime(tr_total['date'], format='%Y-%m-%dT%H:%M:%S.%fZ', errors='coerce')

tr_total['year'] = tr_total['date'].dt.to_period('Y') # Format: YYYY
tr_total['year_month'] = tr_total['date'].dt.to_period('M')  # Format: YYYY-MM
tr_total['year_month_day'] = tr_total['date'].dt.to_period('D')  # Format: YYYY-MM-DD
tr_total['year_month_day_hour'] = tr_total['date'].dt.to_period('h')  # Format: YYYY-MM-DD HH
monthly_data = tr_total.groupby(['customer_id', 'year_month'])['product_id'].count().reset_index().rename(columns = {"product_id" : "transactions"})

In [35]:
all_dates = pd.period_range(start=monthly_data['year_month'].min(),
                            end=monthly_data['year_month'].max(),
                            freq='M')

all_combinations = pd.MultiIndex.from_product(
    [monthly_data['customer_id'].unique(), all_dates],
    names=['customer_id', 'year_month']
).to_frame(index=False)

# Merge with the original data to fill in missing months
complete_data = all_combinations.merge(monthly_data, on=['customer_id', 'year_month'], how='left')
complete_data['transactions'] = complete_data['transactions'].fillna(0)

In [ ]:
complete_data.tail()

In [ ]:
complete_data[complete_data["customer_id"] == 1001614].plot("year_month", "transactions")

In [ ]:
complete_data[complete_data["customer_id"] == 9997881].plot("year_month", "transactions")

In [36]:
window_size = 3
targets = []

for customer_id in complete_data['customer_id'].unique():
    customer_data = complete_data[complete_data['customer_id'] == customer_id]
    
    for i in range(len(customer_data) - window_size + 1):
        target_sum = customer_data['transactions'].iloc[i:i+window_size].sum()
        
        targets.append({
            'customer_id': customer_id,
            'start_year': customer_data['year_month'].iloc[i].year,
            'start_month': customer_data['year_month'].iloc[i].month,
            'end_year': customer_data['year_month'].iloc[i+window_size-1].year,
            'end_month': customer_data['year_month'].iloc[i+window_size-1].month,
            'transactions': target_sum
        })

targets_df = pd.DataFrame(targets)
targets_df = targets_df.sort_values(by=['customer_id', 'start_year', 'start_month'])

In [ ]:
targets_df.head()

In [37]:
for lag in range(3, 31, 3):
    targets_df[f'lag_{lag}'] = targets_df.groupby('customer_id')['transactions'].shift(lag).fillna(0)

### transactions lags 

In [ ]:
autocorrelations = []
nlags = 40

# Group by customer_id and calculate autocorrelation for each customer
for customer, group in targets_df.groupby('customer_id'):
    transactions = group['transactions'].values
    if len(transactions) > 1: 
        acf_values = acf(transactions, nlags=nlags, fft=False)  # Calculate ACF for the first lags
        autocorrelations.append(acf_values)

# Average autocorrelation across customers
average_autocorrelation = np.mean(autocorrelations, axis=0)

plt.figure(figsize=(10, 6))
plt.bar(range(3, len(average_autocorrelation), 3), average_autocorrelation[3::3])
plt.title('Average Autocorrelation Across Customers')
plt.xlabel('Lag')
plt.ylabel('Autocorrelation')
plt.show()

In [ ]:
targets_df.head()

In [ ]:
plt.figure(figsize=(20, 12))

# Create the heatmap with annotations
sns.heatmap(
    targets_df.corr(), 
    annot=True,        
    fmt=".2f",          
    cmap="coolwarm",    
    annot_kws={"size": 12}, 
    linewidths=0.5,     
    square=True         
)

plt.title("Correlation Heatmap", fontsize=16)
plt.show()

### Train/test split

In [38]:
train_year = 2019
train_month = 1

train = targets_df[
    (targets_df['end_year'] < train_year) |
    ((targets_df['end_year'] == train_year) & (targets_df['end_month'] <= train_month))
]

test = targets_df[
    (targets_df['start_year'] == train_year) &
    (targets_df['start_month'] == train_month + 1)
]

In [39]:
train.columns, train.shape, test.shape

(Index(['customer_id', 'start_year', 'start_month', 'end_year', 'end_month',
        'transactions', 'lag_3', 'lag_6', 'lag_9', 'lag_12', 'lag_15', 'lag_18',
        'lag_21', 'lag_24', 'lag_27', 'lag_30'],
       dtype='object'),
 (46046, 16),
 (2002, 16))

In [40]:
feat_cols = ['customer_id', 'start_year', 'start_month', 'end_year', 'end_month',
            'lag_3', 'lag_6', 'lag_9', 'lag_12', 'lag_15', 'lag_18', 'lag_21']

In [196]:
# Features and target for training
X_train = train[feat_cols]
y_train = train['transactions']

# Features and target for testing
X_test = test[feat_cols]
y_test = test['transactions']

In [197]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((46046, 12), (46046,), (2002, 12), (2002,))

In [ ]:
X_test.head()

### Benchmark

* mean of previous customer transactions
* Previous three months transactions 

In [ ]:
mean_transactions = train.groupby('customer_id')['transactions'].mean().reset_index()
mean_transactions.rename(columns={'transactions': 'mean_transactions'}, inplace=True)
test_with_mean = test.merge(mean_transactions, on='customer_id', how='left')

In [ ]:
y_pred_benchmark = test_with_mean['mean_transactions']

# Compute evaluation metrics for the benchmark
mae_benchmark = mean_absolute_error(y_test, y_pred_benchmark)
rmse_benchmark = root_mean_squared_error(y_test, y_pred_benchmark)

# Print the benchmark results
print("Benchmark Results (Mean Prediction):")
print(f"Mean Absolute Error (MAE): {mae_benchmark:.2f}")

In [ ]:
last_train_values = train[(train["end_year"] == 2019) & (train["end_month"] == 1)]["transactions"]

# Compute evaluation metrics for the benchmark
mae_benchmark_2 = mean_absolute_error(y_test, last_train_values)
rmse_benchmark_2 = root_mean_squared_error(y_test, last_train_values)

# Print the benchmark results
print("Benchmark Results (Mean Prediction):")
print(f"Mean Absolute Error (MAE): {mae_benchmark_2:.2f}")

### Feature selection

In [ ]:
lasso = LassoCV(cv=5, random_state=42)  # cv=5 for 5-fold cross-validation
lasso.fit(X_train, y_train)

# Get the indices of features with non-zero coefficients
selected_features = np.where(lasso.coef_ != 0)[0]

print(f"Number of selected features: {len(selected_features)}")
if hasattr(X_train, 'columns'):
    print("Selected features:", X_train.columns[selected_features])  

In [ ]:
X_train_small, X_val, y_train_small, y_val = train_test_split(X_train, y_train, test_size=0.3, random_state=42)

rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train_small, y_train_small)
result = permutation_importance(rf, X_val, y_val, n_repeats=5, random_state=42, n_jobs=-1)
importance_df = pd.DataFrame({
    'feature': X_train.columns,  # Replace with feature names if available
    'importance_mean': result.importances_mean,
    'importance_std': result.importances_std
})

importance_df = importance_df.sort_values('importance_mean', ascending=False)
plt.figure(figsize=(10, 6))
plt.barh(importance_df['feature'], importance_df['importance_mean'], xerr=importance_df['importance_std'])
plt.xlabel("Permutation Importance")
plt.ylabel("Feature")
plt.title("Random Forest Permutation Importance (Validation Set)")
plt.gca().invert_yaxis() 
plt.show()

selected_features = importance_df['feature'][:10]
rf_final = RandomForestRegressor(n_estimators=100, random_state=42)
rf_final.fit(X_train[selected_features], y_train)
test_score = rf_final.score(X_test[selected_features], y_test)
print(f"Test set R^2 score: {test_score:.4f}")

In [198]:
selected_features = ["customer_id","start_year", "start_month", "end_year", "end_month", "lag_3", "lag_6", "lag_9", "lag_12", "lag_15"]
X_train_selected = X_train.loc[:, selected_features]  
X_test_selected = X_test.loc[:, selected_features]  

### Ridge regression

In [43]:
ridge = Ridge(alpha = 1)
ridge.fit(X_train_selected, y_train)
y_train_pred = ridge.predict(X_train_selected)
y_test_pred = ridge.predict(X_test_selected)

y_train_pred = np.clip(y_train_pred, 0, None)
y_test_pred = np.clip(y_test_pred, 0, None)

# Calculate MAE
train_mae = mean_absolute_error(y_train, y_train_pred)
test_mae = mean_absolute_error(y_test, y_test_pred)

# Print the results
print("Training Results:")
print(f"MAE: {train_mae:.7f}")

print("Testing Results:")
print(f"MAE: {test_mae:.7f}")

Training Results:
MAE: 40.7847780
Testing Results:
MAE: 22.0856233


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=2.9309e-18): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T


In [ ]:
pd.concat([X_test, pd.Series(y_test, name = "test"), pd.Series(y_test_pred,  index=X_test.index, name="predictions"), abs(y_test - y_test_pred)], axis = 1).rename(columns = {"transactions" : "error"}).sort_values("error", ascending=False)

In [ ]:
complete_data[complete_data["customer_id"] == 4893343].plot("year_month", "transactions")

In [ ]:
complete_data[complete_data["customer_id"] == 6026638].plot("year_month", "transactions")

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

ax1.scatter(y_train, y_train_pred, alpha=0.5, color='blue', label='Predictions')
ax1.plot([y_train.min(), y_train.max()], [y_train.min(), y_train.max()], 'k--', lw=2, label='Ideal Fit')
ax1.set_xlabel("Ground Truth (Training)")
ax1.set_ylabel("Predictions (Training)")
ax1.set_title("Training: Predictions vs. Ground Truth")
ax1.legend()

ax2.scatter(y_test, y_test_pred, alpha=0.5, color='red', label='Predictions')
ax2.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=2, label='Ideal Fit')
ax2.set_xlabel("Ground Truth (Testing)")
ax2.set_ylabel("Predictions (Testing)")
ax2.set_title("Testing: Predictions vs. Ground Truth")
ax2.legend()

# Show the plot
plt.tight_layout()
plt.show()

### Random Forest

In [ ]:
X_train_selected.shape, X_test_selected.shape

In [ ]:
model = RandomForestRegressor(
            criterion="absolute_error",
            n_estimators = 80,
            max_depth = 5,
            max_features="sqrt",
            min_samples_split = 2,
            n_jobs = -1,
            random_state=42,
            verbose = 1,
        )

model.fit(X_train_selected, y_train)

y_train_pred = model.predict(X_train_selected)
y_test_pred = model.predict(X_test_selected)
train_mae = mean_absolute_error(y_train, y_train_pred)
test_mae = mean_absolute_error(y_test, y_test_pred)

print(f"Training MAE: {train_mae:.4f}")
print(f"Test MAE: {test_mae:.4f}") # 16.32 with 100, 16.4 with 120

In [ ]:
model.feature_importances_

In [ ]:
most_important_tree_index = np.argmax([tree.tree_.impurity[0] for tree in model.estimators_])
most_important_tree = model.estimators_[most_important_tree_index]

In [ ]:
# Visualize the most important tree
tree_dot = export_graphviz(
    most_important_tree,
    out_file=None,
    feature_names=X_train_selected.columns,
    filled=True,
    rounded=True,
    special_characters=True,
    impurity=True
)

# Render and display the tree inline in the notebook
graph = Source(tree_dot)
graph  # This will display the tree directly in the notebook


### Boost models

In [49]:
def objective(trial):
    # Sample hyperparameters
    params = {
        'objective': "reg:absoluteerror",  # Use custom MAE objective with alpha
        'eval_metric': 'mae',  # Mean absolute error metric for evaluation
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 1e-1, log = True),
        'tree_method': 'hist',  # Use 'hist' for faster training, or 'gpu_hist' for GPU acceleration
        'device': 'cuda',  # Use GPU if available
        'n_jobs': -1,
        'random_state': 42,
    }
    num_boost_round = trial.suggest_int('num_boost_round', 10, 300)
    ts_split = TimeSeriesSplit(n_splits=5) 
    mae_scores = []  # To store MAE scores for each fold
    for train_idx, val_idx in ts_split.split(X_train_selected, y_train):
        X_train_fold, X_val_fold = X_train_selected.iloc[train_idx], X_train_selected.iloc[val_idx]
        y_train_fold, y_val_fold = y_train.iloc[train_idx], y_train.iloc[val_idx]
        
        dtrain = xgb.DMatrix(X_train_fold, label=y_train_fold)
        dval = xgb.DMatrix(X_val_fold, label=y_val_fold)
        
        model_xgb = xgb.train(params, dtrain, evals=[(dval, 'eval')], num_boost_round=num_boost_round, early_stopping_rounds=10, verbose_eval=False)
        y_val_pred = model_xgb.predict(dval)
        
        mae = mean_absolute_error(y_val_fold, y_val_pred)
        mae_scores.append(mae)
    
    # Return the mean MAE across all folds
    return np.mean(mae_scores)  # Optuna minimizes the objective, so we return the MAE

# Create a study object and optimize it
study = optuna.create_study(direction='minimize')  # Minimize MAE
study.optimize(objective, n_trials=50)  # Run 50 trials

# Print the best hyperparameters found
print(f"Best Hyperparameters: {study.best_params}")

[I 2025-01-26 18:42:37,637] A new study created in memory with name: no-name-c6626dfb-b5b3-428d-a6fc-1bc44b24e219
[I 2025-01-26 18:42:40,453] Trial 0 finished with value: 30.219685552228633 and parameters: {'max_depth': 6, 'colsample_bytree': 0.6206627670760091, 'min_child_weight': 10, 'learning_rate': 0.03500649452275317, 'num_boost_round': 266}. Best is trial 0 with value: 30.219685552228633.
[I 2025-01-26 18:42:43,622] Trial 1 finished with value: 37.931849451668725 and parameters: {'max_depth': 7, 'colsample_bytree': 0.5201628082958782, 'min_child_weight': 8, 'learning_rate': 0.0027731782274070223, 'num_boost_round': 163}. Best is trial 0 with value: 30.219685552228633.
[I 2025-01-26 18:42:44,334] Trial 2 finished with value: 39.88007588526343 and parameters: {'max_depth': 5, 'colsample_bytree': 0.6421023405469795, 'min_child_weight': 1, 'learning_rate': 0.005982018743932518, 'num_boost_round': 41}. Best is trial 0 with value: 30.219685552228633.
[I 2025-01-26 18:42:49,379] Trial 3

Best Hyperparameters: {'max_depth': 5, 'colsample_bytree': 0.7841099688463887, 'min_child_weight': 10, 'learning_rate': 0.021779506146773927, 'num_boost_round': 277}


In [58]:
best_params = study.best_params
best_max_depth = best_params['max_depth']
best_colsample_bytree = best_params['colsample_bytree']
best_min_child_weight = best_params['min_child_weight']
best_learning_rate = best_params['learning_rate']
best_num_boost_round = best_params['num_boost_round']

params = {
    'objective': "reg:absoluteerror",  # Use custom MAE objective with alpha
    'eval_metric': 'mae',  # Mean absolute error metric for evaluation
    'max_depth': best_max_depth,
    'colsample_bytree': best_colsample_bytree,
    'min_child_weight': best_min_child_weight,
    'learning_rate': best_learning_rate,
    'tree_method': 'hist',  # Use 'hist' for faster training, or 'gpu_hist' for GPU acceleration
    'device': 'cuda',  # Use GPU if available
    'n_jobs': -1,
    'random_state': 42,
}

dtrain = xgb.DMatrix(X_train_selected, label=y_train)
dtest = xgb.DMatrix(X_test_selected, label=y_test)

model_xgb = xgb.train(
    params,
    dtrain,
    num_boost_round=best_num_boost_round,  # Use the best num_boost_round from Optuna
    evals=[(dtrain, 'train')],
    verbose_eval=False
)

y_test_pred = model_xgb.predict(dtest)
test_mae = mean_absolute_error(y_test, y_test_pred)
print(f"Test MAE: {test_mae:.4f}")

Test MAE: 16.9417


### Neural net

In [152]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

class MLPRegressor(nn.Module):
    def __init__(self, input_dim, hidden_dim=100, dropout_rate=0.2):
        super(MLPRegressor, self).__init__()
        
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.dropout1 = nn.Dropout(dropout_rate)  # Dropout layer to reduce overfitting
        
        self.fc2 = nn.Linear(hidden_dim, 2*hidden_dim)
        self.fc3 = nn.Linear(2*hidden_dim, hidden_dim)
        self.fc4 = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.dropout1(x)
        
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        
        x = self.fc4(x) 
        x = torch.relu(x)  # Apply ReLU to ensure non-negative output
        return x

Using device: cuda


In [96]:
train_selected = pd.concat([X_train_selected, y_train], axis = 1)

In [219]:
train_year = 2018
train_month = 10

# Split the data based on time - use training data until June 2018 and validation data from July 2018
train_2 = train_selected[
    (train_selected['end_year'] < train_year) |
    ((train_selected['end_year'] == train_year) & (train_selected['end_month'] <= train_month))
]

valid = train_selected[
    (train_selected['start_year'] > train_year) |
    ((train_selected['start_year'] == train_year) & (train_selected['start_month'] > train_month))
]

In [222]:
X_train = train_2.drop(columns=['transactions'])  
y_train = train_2['transactions']
X_valid = valid.drop(columns=['transactions'])
y_valid = valid['transactions']

In [232]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)
X_test_scaled = scaler.transform(X_test_selected)

In [223]:
X_train.shape, y_train.shape, X_valid.shape, y_valid.shape

((40040, 10), (40040,), (2002, 10), (2002,))

In [235]:
# Prepare the data as PyTorch tensors
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).to(device)

X_valid_tensor = torch.tensor(X_valid_scaled, dtype=torch.float32).to(device)
y_valid_tensor = torch.tensor(y_valid.values, dtype=torch.float32).to(device)

X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32).to(device)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).to(device)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True)
valid_dataset = TensorDataset(X_valid_tensor, y_valid_tensor)
valid_loader = DataLoader(valid_dataset, batch_size=512, shuffle=False)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=512, shuffle=False)

In [270]:
input_dim = X_train_tensor.shape[1]
model = MLPRegressor(input_dim,
                    hidden_dim=100, 
                    dropout_rate=0.2).to(device)

# Loss and optimizer (using MAE as loss function)
criterion = nn.L1Loss()  # MAE Loss
optimizer = optim.Adam(model.parameters(), lr=3*10**-5)

# Early stopping parameters
patience = 5
best_valid_loss = float('inf')
epochs_without_improvement = 0

num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs.squeeze(), labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    model.eval()
    valid_loss = 0.0
    with torch.no_grad():
        for inputs, labels in valid_loader:
            outputs = model(inputs)
            loss = criterion(outputs.squeeze(), labels)
            valid_loss += loss.item()

    # Average losses for the epoch
    train_loss = running_loss / len(train_loader)
    valid_loss = valid_loss / len(valid_loader)
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Valid Loss: {valid_loss:.4f}")
    # Early stopping check
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        epochs_without_improvement = 0
        # Save the model (optional)
        torch.save(model.state_dict(), "best_model.pth")
    else:
        epochs_without_improvement += 1
    if epochs_without_improvement >= patience:
        print(f"Early stopping at epoch {epoch+1}")
        break

Epoch [1/100], Train Loss: 46.4427, Valid Loss: 46.1950
Epoch [2/100], Train Loss: 46.0001, Valid Loss: 46.1448
Epoch [3/100], Train Loss: 46.5258, Valid Loss: 46.0494
Epoch [4/100], Train Loss: 45.8298, Valid Loss: 45.8798
Epoch [5/100], Train Loss: 45.7122, Valid Loss: 45.6033
Epoch [6/100], Train Loss: 45.5316, Valid Loss: 45.1796
Epoch [7/100], Train Loss: 45.0203, Valid Loss: 44.5734
Epoch [8/100], Train Loss: 44.7417, Valid Loss: 43.7651
Epoch [9/100], Train Loss: 44.0843, Valid Loss: 42.7379
Epoch [10/100], Train Loss: 43.4654, Valid Loss: 41.5267
Epoch [11/100], Train Loss: 42.5227, Valid Loss: 40.1742
Epoch [12/100], Train Loss: 41.7892, Valid Loss: 38.6670
Epoch [13/100], Train Loss: 40.8229, Valid Loss: 37.1567
Epoch [14/100], Train Loss: 40.3526, Valid Loss: 35.8261
Epoch [15/100], Train Loss: 39.4676, Valid Loss: 34.7089
Epoch [16/100], Train Loss: 39.2531, Valid Loss: 33.8404
Epoch [17/100], Train Loss: 38.7523, Valid Loss: 33.1087
Epoch [18/100], Train Loss: 38.0561, Val

In [271]:
model.load_state_dict(torch.load("best_model.pth"))
model.eval()
y_pred = []
y_true = []
with torch.no_grad():
    for inputs, labels in test_loader:
        print(inputs.shape)
        outputs = model(inputs)
        y_pred.extend(outputs.squeeze().cpu().numpy())
        y_true.extend(labels.cpu().numpy())

mae = mean_absolute_error(y_true, y_pred)
print(f"Test MAE: {mae:.4f}")

torch.Size([512, 10])
torch.Size([512, 10])
torch.Size([512, 10])
torch.Size([466, 10])
Test MAE: 17.6504


<ipython-input-271-826e3d7ea663>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("best_model.pth"))
